In [1]:
import pp
import numpy as np
from pyPoseidon.dem import *
from pyPoseidon.utils.fix import *
import shapely

## Set

In [2]:
dic={'minlon':-32., # lat/lon window
     'maxlon':-29,
     'minlat':50.5,
     'maxlat':52.5,
     'dpath':'/Users/brey/DATA/GEBCO_2014_2D.nc'}

In [55]:
dic = {'minlon' : 0., # lat/lon window
       'maxlon' : 13.,
       'minlat' : 54.5,
       'maxlat' : 66.5,
       'dpath':'/Users/brey/DATA/GEBCO_2014_2D.nc'}

In [2]:
dic = {'minlon':-35., # lat/lon window
       'maxlon':42.,
       'minlat':25.05,
       'maxlat':76.5,
       'dpath':'/Users/brey/DATA/GEBCO_2014_2D.nc'}

### Create Grid

In [3]:
resolution=.1

## lat,lon grid
ni=int(round((dic['maxlon']-dic['minlon'])/resolution)) #these are cell numbers
nj=int(round((dic['maxlat']-dic['minlat'])/resolution))

dic['maxlon']=dic['minlon']+ni*resolution #adjust max lon to much the grid
dic['maxlat']=dic['minlat']+nj*resolution


lon=np.linspace(dic['minlon'],dic['maxlon'],ni)
lat=np.linspace(dic['minlat'],dic['maxlat'],nj)
x, y = np.meshgrid(lon,lat)

In [4]:
dic.update({'grid_x':x, 'grid_y':y})

In [5]:
#Define the coastline shapefile
shapefile = '/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_%sm_coastline.shp' % \
                    {'l':110, 'i':50, 'h':10}['i']

In [6]:
dem = gebco(**dic) # get bathymetry


extracting dem from /Users/brey/DATA/GEBCO_2014_2D.nc

.. interpolating on grid ..

dem done


In [7]:
dem.Dataset

<xarray.Dataset>
Dimensions:  (dlat: 240, dlon: 360, i: 240, ilat: 20, ilon: 30, j: 360, k: 20, l: 30)
Coordinates:
  * dlat     (dlat) float64 50.5 50.51 50.52 50.53 50.54 50.55 50.55 50.56 ...
  * dlon     (dlon) float64 -32.0 -32.0 -31.99 -31.98 -31.97 -31.96 -31.95 ...
  * ilat     (ilat) float64 50.5 50.61 50.71 50.82 50.92 51.03 51.13 51.24 ...
  * ilon     (ilon) float64 -32.0 -31.9 -31.79 -31.69 -31.59 -31.48 -31.38 ...
Dimensions without coordinates: i, j, k, l
Data variables:
    dlats    (i, j) float64 50.5 50.5 50.5 50.5 50.5 50.5 50.5 50.5 50.5 ...
    val      (dlat, dlon) float32 -3319.0 -3270.0 -3213.0 -3162.0 -3111.0 ...
    dlons    (i, j) float64 -32.0 -32.0 -31.99 -31.98 -31.97 -31.96 -31.95 ...
    ival     (ilat, ilon) float32 -3319.0 -2866.0 -3364.0 -3624.0 -3115.0 ...
    ilats    (k, l) float64 50.5 50.5 50.5 50.5 50.5 50.5 50.5 50.5 50.5 ...
    ilons    (k, l) float64 -32.0 -31.9 -31.79 -31.69 -31.59 -31.48 -31.38 ...

In [8]:
#define coastline
    
shp = gp.GeoDataFrame.from_file(shapefile)

           
xp = dem.Dataset.ilons.values
yp = dem.Dataset.ilats.values
    
    
#put them all in a list
ls=[]
for i in range(shp.shape[0]):
    il = shp.loc[i,'geometry']
    try:
        #print len(il)
        for k in range(len(list(il.geoms))): # for MultiLineStrings
            ls.append(list(il.geoms)[k])
    except:
        ls.append(il)
            
sall = shapely.geometry.MultiLineString(ls) # join
c = shapely.ops.linemerge(sall) #merge
    
#Select the Line Strings that correspond to our grid
#create a polygon of the grid
grp=shapely.geometry.Polygon([(xp.min(),yp.min()),(xp.min(),yp.max()),(xp.max(),yp.max()),(xp.max(),yp.min())])    
    
cl=[] #initialize
#add Polygons if they intersect with the grid
for i in range(len(c)):
    z = shapely.geometry.Polygon(c[i])
    if z.intersects(grp): 
        cl.append(c[i])
    
cll = shapely.geometry.MultiLineString(cl) #join them into a Multiline
cg = shapely.ops.linemerge(cll) #merge parts if possible
    


In [9]:
def loop0(xp,yp,cg):
#check if the grid polygons intersect the shoreline
    gps = []
    igps = []
    for i in range(0,xp.shape[0]-1):
        for j in range(0,xp.shape[1]-1):
            p = shapely.geometry.Polygon([(xp[i,j],yp[i,j]),(xp[i+1,j],yp[i+1,j]),(xp[i+1,j+1],yp[i+1,j+1]),(xp[i,j+1],yp[i,j+1])])
            if not p.intersects(cg): 
                gps.append(p)
                igps.append([i+1,j+1])
                if i == 0 : 
                    igps.append([i,j])
                    igps.append([i,j+1])
                if j == 0 : 
                    igps.append([i,j])
                    igps.append([i+1,j])
    return igps

## One core

In [10]:
#%%timeit 
g1 = loop0(xp,yp,cg)

In [11]:
solo = np.array(g1)

In [12]:
solo.shape

(647, 2)

## PP loop

In [13]:
def loop(xp,yp,cg):
#check if the grid polygons intersect the shoreline
    gps = []
    igps = []
    for i in range(xp.shape[0]-1):
        for j in range(xp.shape[1]-1):
            p = shapely.geometry.Polygon([(xp[i,j],yp[i,j]),(xp[i+1,j],yp[i+1,j]),(xp[i+1,j+1],yp[i+1,j+1]),(xp[i,j+1],yp[i,j+1])])
            if not p.intersects(cg): 
                gps.append(p)
                igps.append([i+1,j+1])
    
    return igps

In [14]:
job_server = pp.Server() 

INFO:pp:Creating server instance (pp-1.6.6)
INFO:pp:Running on Python 2.7.15 darwin
INFO:pp:pp local server started with 8 workers


In [15]:
job_server.get_ncpus()

8

In [16]:
ncpus=8

In [17]:
job_server.set_ncpus(ncpus)

### split the array 

In [18]:
n = xp.shape[0]
l=range(n)
k = ncpus

In [19]:
b = [l[i * (n // k) + min(i, n % k):(i+1) * (n // k) + min(i+1, n % k)] for i in range(k)]


In [20]:
#%%timeit
jobs=[]
for l in range(ncpus):
    extra=b[l][-1]+1 # add one to consider the jump between parts
    part = b[l]+[extra]
    if part[-1] >= xp.shape[0] : part = part[:-1] #control the last element
    xl=xp[part,:]
    yl=yp[part,:]   


    jobs.append(job_server.submit(loop,(xl,yl,cg,),modules=('shapely',)))

ps=jobs[0]()
for l in range(1,ncpus):    
    ps = np.vstack([ps, [[i+b[l][0],j] for [i,j] in jobs[l]()]]) # fix global index by adding the first index of part

INFO:pp:Task 0 started
INFO:pp:Task 1 started
INFO:pp:Task 2 started
INFO:pp:Task 3 started
INFO:pp:Task 4 started
INFO:pp:Task 5 started
INFO:pp:Task 6 started
INFO:pp:Task 7 started


In [33]:
job_server.destroy()

/Users/brey/miniconda3/envs/dev/lib/python2.7/site-packages/pyPoseidon-0.3.0-py2.7-macosx-10.9-x86_64.egg/pyPoseidon/utils/vis.py:131: AccessorRegistrationWarning: registration of accessor <class 'pyPoseidon.utils.vis.splot'> under name 'splot' for type <class 'xarray.core.dataset.Dataset'> is overriding a preexisting attribute with the same name.
  class splot(object):


## Add extra points from boundary


In [21]:
bgps=[]
for [i,j] in ps:
    if i == 1 : 
        bgps.append([i-1,j-1])
        bgps.append([i-1,j])
    if j == 1 : 
        bgps.append([i-1,j-1])
        bgps.append([i,j-1])

In [22]:
ps = np.vstack([ps, bgps]) # final stack

## Compare

In [23]:
sorted(solo[:,0]) == sorted(ps[:,0])

True

In [24]:
sorted(solo[:,1]) == sorted(ps[:,1])

True

In [25]:
#create a mask of all cells not intersecting the shoreline
imask=np.ones(xp.shape, dtype=bool)
for [i,j] in ps:
    imask[i,j]=0                


In [32]:
cg.geom_type

'GeometryCollection'

In [26]:

points=zip(xp.flatten(),yp.flatten())
    
#collect the internal points
xi=[]
yi=[]
if cg.geom_type.startswith('Multi'):
        for i in range(len(cg)):
            z = shapely.geometry.Polygon(cg[i])
            path = mpltPath.Path(zip(z.boundary.xy[0],z.boundary.xy[1]))

            inside = path.contains_points(points)

            if np.sum(inside) > 0:
                X = np.ma.masked_array(xp,mask=np.invert(inside)),
                Y = np.ma.masked_array(yp,mask=np.invert(inside))
                xi.append(X)
                yi.append(Y) 
else:
        z = shapely.geometry.Polygon(cg)
        path = mpltPath.Path(zip(z.boundary.xy[0],z.boundary.xy[1]))

        inside = path.contains_points(points)

        if np.sum(inside) > 0:
            X = np.ma.masked_array(xp,mask=np.invert(inside)),
            Y = np.ma.masked_array(yp,mask=np.invert(inside))
            xi.append(X)
            yi.append(Y) 


    
    #merge the masks 
gmask=np.ones(xi[0][0].shape, dtype=bool)
for i in range(len(xi)):
        gmask = np.logical_and(gmask,xi[i][0].mask)    
    

NotImplementedError: Multi-part geometries do not themselves provide the array interface

In [126]:
        
# merge gmask and imask
tmask =  np.logical_and(np.invert(imask),gmask)
tmask = np.invert(tmask) 


In [127]:
np.array_equal(imask,gmask)

True

In [128]:
np.array_equal(imask,tmask)

False

In [131]:
imask#.sum(), gmask.sum(), tmask.sum()

/Users/brey/miniconda3/envs/dev/lib/python2.7/site-packages/pyPoseidon-0.3.0-py2.7-macosx-10.9-x86_64.egg/pyPoseidon/utils/vis.py:131: AccessorRegistrationWarning: registration of accessor <class 'pyPoseidon.utils.vis.splot'> under name 'splot' for type <class 'xarray.core.dataset.Dataset'> is overriding a preexisting attribute with the same name.
  class splot(object):


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [83]:
nc=50

In [84]:
grid = np.invert(tmask).astype(int).astype(str)
grid = [list(x) for x in grid]
isls = Solution().numIslands(grid)



In [85]:
isls

59

In [86]:
mgrid = np.array(grid).astype(int)
nps = []
for i in range(1, isls + 1):
    nps.append(np.sum(mgrid != i))
    
grs = pd.DataFrame([(val, idx) for (idx, val) in enumerate(nps)],columns=['val','idx'])

grs = grs.sort_values('val').reset_index(drop=True)
    
grs['np']=mgrid.size-grs.val # count number of points in islands
    
ne = grs[grs.np>nc].index.max() + 1



In [87]:
ne

2

In [88]:
mask={}
for i in range(ne):
        b = mgrid.copy()
        vm = np.ma.masked_array(b, b != grs.loc[i,'idx']+1)
        vm[np.invert(vm.mask)] = 1    
        mask.update({str(i):vm})

fmask=np.zeros(xp.shape, dtype=bool) # initiate
